In [1]:
import numpy as np
import pandas as pd

In [2]:
def tick_to_rate(tick_index: int) -> float:
    if tick_index >= 0:
        return (1.0001)**tick_index - 1
    else:
        return 1 - (1.0001)**(-tick_index)

def rate_to_tick(rate: float) -> int:
    if rate >= 0:
        return float(round(np.log(1+rate)/np.log(1.0001),6))
    else:
        return - float(round(np.log(1-rate)/np.log(1.0001),6))
def str_to_ns(string: str):
    """
    Converts a human readable time-delta string into nanoseconds.

    Arguments:
        string: String to convert into nanoseconds. Uses Pandas to do this.

    Examples:
        - "1s" -> 1e9 ns
        - "1min" -> 6e10 ns
        - "00:00:30" -> 3e10 ns
    """
    return pd.to_timedelta(string).to_timedelta64().astype(np.int64)

In [3]:
import inspect
import hashlib
import os
import pickle
from typing import List, Dict, Any, Callable

import numpy as np
import pandas as pd

def datetime_str_to_ns(string: str):
    """
    Takes a datetime written as a string and returns in nanosecond unix timestamp.

    Arguments:
        string: String to convert into nanoseconds. Uses Pandas to do this.
    """
    return pd.Timestamp(string).value


In [11]:
(1-0.05-0.4)*0.1+0.4*0.1095+0.05*0.1082

0.10421

In [17]:
(1 - 0.4 - 0.05)*0.10 + 0.05*0.1095 + 0.4*0.10826403685047903

0.10378061474019161

In [13]:
0.05*0.1095

0.005475000000000001

In [14]:
0.40*0.108264

0.0433056

In [15]:
0.055 + 0.005475 + 0.0433056

0.1037806

In [ ]:
rate_to_tick(0.1095)

1039.146588

In [7]:
tick_to_rate(1050)

0.11070477952901259

In [10]:
datetime_str_to_ns('2024-01-01T00:00:00.000Z')

1704067200000000000

In [4]:
pd.read_pickle('BTC-Binance-2023.pkl')

,Binance,date_time
0,0.000100,2023-01-01T08:00:00.008Z
1,0.000007,2023-01-01T16:00:00.000Z
2,-0.000015,2023-01-02T00:00:00.000Z
3,-0.000044,2023-01-02T08:00:00.008Z
4,0.000027,2023-01-02T16:00:00.015Z
...,...,...
1090,0.000378,2023-12-30T16:00:00.000Z
1091,0.000279,2023-12-31T00:00:00.000Z
1092,0.000346,2023-12-31T08:00:00.001Z
1093,0.000336,2023-12-31T16:00:00.000Z


In [11]:
position = {"COLLATERAL": 50, "SIZE": 100, "FIXRATE": 0.0001}
mkt_open = 0
mkt_close = 365 * str_to_ns("1d")
current_time = 0
swap_interval = 8 * str_to_ns("1h")
last_twap = -3.230998

In [ ]:
mkt_close = 365 * str_to_ns("1d")

In [12]:
def mark_to_market(
    position=None, market_tick: int = None, log: bool = False
) -> int:
    """
    Marks position to market (including cash) based on the market rate from rate_oracle.

    Arguments:
        position:
        log: log this event or not
    """
    # If no position is provided, self evaluating the current position
    if not position:  
        position = position 
        log = True  # Only log the real mark_to_market

    # If market tick is not provided, find market_tick from orderbook directly
    if not market_tick:
        market_tick = last_twap  # Should be like this in the future

    cash = position["COLLATERAL"]
        
    value = (tick_to_rate(market_tick)-position["FIXRATE"])*position["SIZE"]*n_payment()

    cash += value


    return cash

def n_payment():
    return np.ceil((mkt_close - current_time)/swap_interval)

In [13]:
mark_to_market(position=position, market_tick=last_twap, log=True)

3.666625166404451

In [15]:
import math

# 定义位置
position = {
    "COLLATERAL": 50,   # 保证金
    "SIZE": 100,        # 持仓量
    "FIXRATE": 0.0001   # 固定利率
}

# 定义 TWAP
twap = -3.230998

# 定义 tick_to_rate，根据 FakeRateOracle 的返回值
def tick_to_rate(tick: float) -> float:
    # 在您的测试中，FakeRateOracle 返回了 -0.0003231358432291831
    return -0.0003231358432291831

# 定义 rate_normalizer
rate_normalizer = 1

# 定义 n_payment，假设为 1095
n_payment = 1095

# 计算 tick_to_rate(twap)
floating_rate = tick_to_rate(twap)

# 计算 MtM
def calculate_mark_to_market(position, floating_rate: float, rate_normalizer: float, n_payment: int) -> float:
    collateral = position["COLLATERAL"]
    size = position["SIZE"]
    fixrate = position["FIXRATE"]
    
    # 计算值
    value = (floating_rate - fixrate) * rate_normalizer * size * n_payment
    
    # 计算 MtM
    mark_to_market = collateral + value
    
    return mark_to_market

# 进行计算
mark_to_market = calculate_mark_to_market(position, floating_rate, rate_normalizer, n_payment)

# 打印结果
print(f"Mark to Market Value: {mark_to_market:.3f}")


Mark to Market Value: 3.667
